<a href="https://colab.research.google.com/github/Natthaya04/NATTHAYA_Project/blob/main/Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ติดตั้ง library เพิ่ม + เตรียมสภาพแวดล้อมสำหรับใช้ Spark + Pandas + Kaggle

In [ ]:
#Install extra lib(s)
!pip install -q xlrd
!pip install -q kaggle
!pip install -q kora

import kora
import pandas as pd

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import *

ติดตั้ง/อัปเกรด library opendatasets เพื่อใช้โหลด dataset จาก Kaggle

In [ ]:
%pip install opendatasets --upgrade

ติดตั้งไลบรารี googletrans รุ่น 4.0.0-rc1 ใช้สำหรับแปลภาษาใน Python

In [ ]:
!pip install googletrans==4.0.0-rc1


โหลด dataset จาก Kaggle (YouTube trending videos) แล้ว เก็บไว้ใน mypath

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/datasnaek/youtube-new/data", "/content/mypath/")

ตรวจสอบว่าในโฟลเดอร์มีไฟล์อะไรบ้าง

In [ ]:
import os

# Assuming the path is the same as defined in cell Vy1GNswuqOy5
path = "/content/mypath/youtube-new"
files = os.listdir(path)
for f in files:
    print(f)

In [ ]:
import json

# โหลดไฟล์ JSON
with open('/content/mypath/youtube-new/CA_category_id.json', 'r', encoding='utf-8') as f:
    data = json.load(f)  # data is a dict

# Access the list of items and print the first 5 titles
for video in data['items'][:5]:
    print(video['snippet']['title'])

In [ ]:
import json
import pandas as pd

# โหลด JSON
with open('/content/mypath/youtube-new/CA_category_id.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Process the 'items' list to create a list of dictionaries with relevant data
categories = []
for item in data['items']:
    category_id = item['id']
    category_title = item['snippet']['title']
    categories.append({'category_id': category_id, 'category_title': category_title})

# แปลงเป็น DataFrame
df_category = pd.DataFrame(categories)

# ดู column
print(df_category.columns)

# ดู category ที่มี
print(df_category['category_title'].unique())

# Categoory name และ idทั้งหมด
print(df_category)

# บันทึกเป็น CSV พร้อมรองรับภาษาไทย
df_category.to_csv("df_category.csv", index=False, encoding='utf-8-sig')
print("Saved df_category.csv successfully!")


อ่านไฟล์ CSV จาก mypath ด้วย PySpark เป็น DataFrame

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("YouTubeAnalysis").getOrCreate()

file_path = "/content/mypath/youtube-new" # Corrected the file path

df_input = spark.read.options(header="true", inferschema="true").csv(file_path)

df_input.printSchema()

display(df_input)

ข้อมูล United States

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube US Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/USvideos.csv"

# Read the CSV into df_us

df_us = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_us.printSchema()

# แสดงตัวอย่างข้อมูล
df_us.show(20)


US Cleaning

In [ ]:
print(f"จำนวนแถว: {df_us.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd


# Initialize Spark Session (if not already initialized)
spark = SparkSession.builder \
    .appName("YouTube US Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/USvideos.csv"

# Read the CSV into df_us
df_us = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# เลือกเฉพาะ title ที่เป็นภาษาอังกฤษ
df_us_clean = df_us.filter(
    col("title").rlike("[A-Za-z]")
)
# กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น (0-9)
df_us_clean = df_us_clean.filter(
    col("views").rlike("^[0-9]+$") &
    col("likes").rlike("^[0-9]+$") &
    col("dislikes").rlike("^[0-9]+$")
)

# ลบแถวที่มีค่าว่างในคอลัมน์สำคัญ
df_us_clean = df_us_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_us_clean = df_us_clean.dropDuplicates()

# จัดลำดับตาม views จากมากไปน้อย
df_us_clean = df_us_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_us_clean.show(20, truncate=False)

# แปลงเป็น Pandas และบันทึกเป็น CSV
df_us_clean = df_us_clean.toPandas()
df_us_clean.to_csv("df_us_clean.csv", index=False)

# จำนวนแถวใน
print(f"จำนวนแถว: {df_us_clean.count()}")

เพิ่ม Weeky, Hour, Title_langth, Tags_count, Like_dislike_ratio, day_to_trending

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube US Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/USvideos.csv"

# Read CSV
df_us = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# --- Clean ด้วย Spark ---
df_us_clean_spark = df_us.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_us_clean = df_us_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_us_clean[c] = pd.to_numeric(df_us_clean[c], errors='coerce')

# แปลง datetime
df_us_clean['publish_time'] = pd.to_datetime(df_us_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_us_clean['trending_date'] = pd.to_datetime(df_us_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_us_clean['title_length'] = df_us_clean['title'].astype(str).apply(len)
df_us_clean['tags_count'] = df_us_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_us_clean['days_to_trending'] = (df_us_clean['trending_date'].dt.normalize() - df_us_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_us_clean['weekday'] = df_us_clean['publish_time'].dt.day_name()
df_us_clean['hour'] = df_us_clean['publish_time'].dt.hour

# Save CSV
df_us_clean.to_csv("df_us_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_us_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_us_clean)}")


ข้อมูล Canada



In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube CA Data") \
    .getOrCreate()

#ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/CAvideos.csv"

# Read the CSV into df_ca
df_ca = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)


# ดู schema
df_ca.printSchema()

#แสดงตัวอย่างข้อมูล
df_ca.show(20)

CA Cleaning

In [ ]:
print(f"จำนวนแถว: {df_ca.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd

# Initialize Spark Session (if not already initialized)
spark = SparkSession.builder \
    .appName("YouTube CA Data") \
    .getOrCreate()

#ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/CAvideos.csv"

# Read the CSV into df_ca
df_ca = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_ca_clean = df_ca.filter(
    (col("views").cast("string").rlike("^[0-9]+$")) &
    (col("likes").cast("string").rlike("^[0-9]+$")) &
    (col("dislikes").cast("string").rlike("^[0-9]+$"))
)
# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_ca_clean = df_ca_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_ca_clean = df_ca_clean.dropDuplicates()

# จัดลำดับตาม views จากมากไปน้อย
df_ca_clean = df_ca_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_ca_clean.show(20, truncate=False)

# แปลงเป็น Pandas และบันทึกเป็น CSV
df_ca_clean = df_ca_clean.toPandas()
df_ca_clean.to_csv("df_ca_clean.csv", index=False)

# จำนวนแถวใน
print(f"จำนวนแถว: {df_ca_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube CA Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/CAvideos.csv"

# Read CSV
df_ca = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# --- Clean ด้วย Spark ---
df_ca_clean_spark = df_ca.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_ca_clean = df_ca_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_ca_clean[c] = pd.to_numeric(df_ca_clean[c], errors='coerce')

# แปลง datetime
df_ca_clean['publish_time'] = pd.to_datetime(df_ca_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_ca_clean['trending_date'] = pd.to_datetime(df_ca_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_ca_clean['title_length'] = df_ca_clean['title'].astype(str).apply(len)
df_ca_clean['tags_count'] = df_ca_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_ca_clean['days_to_trending'] = (df_ca_clean['trending_date'].dt.normalize() - df_ca_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_ca_clean['weekday'] = df_ca_clean['publish_time'].dt.day_name()
df_ca_clean['hour'] = df_ca_clean['publish_time'].dt.hour

# Save CSV
df_ca_clean.to_csv("df_ca_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_ca_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_ca_clean)}")


ข้อมูล Germany

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube DE Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/DEvideos.csv"


# Read the CSV into df_de

df_de = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_de.printSchema()

# แสดงตัวอย่างข้อมูล
df_de.show(20)


DE Cleaning

In [ ]:
print(f"จำนวนแถว: {df_de.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
# กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_de_clean = df_de.filter(
    (col("views").cast("string").rlike("^[0-9]+$")) &
    (col("likes").cast("string").rlike("^[0-9]+$")) &
    (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_de_clean = df_de_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_de_clean = df_de_clean.dropDuplicates()

# จัดลำดับตาม views จากน้อยไปมาก
df_de_clean = df_de_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_de_clean.show(20, truncate=False)

# แปลงเป็น Pandas และบันทึกเป็น CSV
df_de_clean = df_de_clean.toPandas()
df_de_clean.to_csv("df_de_clean.csv", index=False)

# จำนวนแถวใน Pandas
print(f"จำนวนแถว: {df_de_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube DE Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/DEvideos.csv"

# Read CSV
df_de = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# --- Clean ด้วย Spark ---
df_de_clean_spark = df_de.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_de_clean = df_de_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_de_clean[c] = pd.to_numeric(df_de_clean[c], errors='coerce')

# แปลง datetime
df_de_clean['publish_time'] = pd.to_datetime(df_de_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_de_clean['trending_date'] = pd.to_datetime(df_de_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_de_clean['title_length'] = df_de_clean['title'].astype(str).apply(len)
df_de_clean['tags_count'] = df_de_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_de_clean['days_to_trending'] = (df_de_clean['trending_date'].dt.normalize() - df_de_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_de_clean['weekday'] = df_de_clean['publish_time'].dt.day_name()
df_de_clean['hour'] = df_de_clean['publish_time'].dt.hour

# Save CSV
df_de_clean.to_csv("df_de_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_de_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_de_clean)}")


In [ ]:
import pandas as pd
from googletrans import Translator

# โหลด CSV
df_de_clean = pd.read_csv("/content/df_de_clean.csv")

# เลือก Top trending 100 แถว (ตาม views สูงสุด)
df_de_clean = df_de_clean.sort_values(by='views', ascending=False).head(100)

# สร้าง Translator
translator = Translator()

# ฟังก์ชันแปลภาษา พร้อมจัดการ error
def translate_text(text):
    try:
        return translator.translate(str(text), dest='en').text
    except:
        return str(text)

# แปล title
df_de_clean['title_en'] = df_de_clean['title'].apply(translate_text)

# บันทึก CSV
df_de_clean.to_csv("/content/df_de_clean.csv", index=False)

# ดูตัวอย่างผลลัพธ์
display(df_de_clean.head(20))

ข้อมูล France


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube FR Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/FRvideos.csv"


# Read the CSV into df_fr

df_fr = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_fr.printSchema()

# แสดงตัวอย่างข้อมูล
df_fr.show(20)


FR Cleaning


In [ ]:
print(f"จำนวนแถว: {df_fr.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
# กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_fr_clean  = df_fr.filter(
    (col("views").cast("string").rlike("^[0-9]+$")) &
    (col("likes").cast("string").rlike("^[0-9]+$")) &
    (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_fr_clean = df_fr_clean .dropna()

# ลบแถวซ้ำทั้งหมด
df_fr_clean = df_fr_clean.dropDuplicates()

# จัดลำดับตาม views จากน้อยไปมาก
df_fr_clean = df_fr_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_fr_clean.show(20, truncate=False)

# แปลงเป็น Pandas และบันทึกเป็น CSV
df_fr_clean = df_fr_clean.toPandas()
df_fr_clean.to_csv("df_fr_clean.csv", index=False)

# จำนวนแถวใน Pandas
print(f"จำนวนแถว: {df_fr_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube FR Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/FRvideos.csv"

# Read CSV
# --- Clean ด้วย Spark ---
df_fr_clean_spark = df_fr.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_fr_clean = df_fr_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_fr_clean[c] = pd.to_numeric(df_fr_clean[c], errors='coerce')

# แปลง datetime
df_fr_clean['publish_time'] = pd.to_datetime(df_fr_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_fr_clean['trending_date'] = pd.to_datetime(df_fr_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_fr_clean['title_length'] = df_fr_clean['title'].astype(str).apply(len)
df_fr_clean['tags_count'] = df_fr_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_fr_clean['days_to_trending'] = (df_fr_clean['trending_date'].dt.normalize() - df_fr_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_fr_clean['weekday'] = df_fr_clean['publish_time'].dt.day_name()
df_fr_clean['hour'] = df_fr_clean['publish_time'].dt.hour

# Save CSV
df_fr_clean.to_csv("df_fr_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_fr_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_fr_clean)}")


ข้อมูล United Kingdom

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube GB Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/GBvideos.csv"


# Read the CSV into df_gb

df_gb = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_gb.printSchema()

# แสดงตัวอย่างข้อมูล
df_gb.show(20)


GB Cieaning

In [ ]:
print(f"จำนวนแถว: {df_gb.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
# กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_gb_clean  = df_gb.filter(
    (col("views").cast("string").rlike("^[0-9]+$")) &
    (col("likes").cast("string").rlike("^[0-9]+$")) &
    (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_gb_clean = df_gb_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_gb_clean = df_gb_clean.dropDuplicates()

# จัดลำดับตาม views จากน้อยไปมาก
df_gb_clean = df_gb_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_gb_clean.show(20, truncate=False)

# แปลงเป็น Pandas และบันทึกเป็น CSV
df_gb_clean = df_gb_clean.toPandas()
df_gb_clean.to_csv("df_gb_clean.csv", index=False)

# จำนวนแถวใน Pandas
print(f"จำนวนแถว: {df_gb_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube GB Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/GBvideos.csv"

# Read CSV
# --- Clean ด้วย Spark ---
df_gb_clean_spark = df_gb.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_gb_clean = df_gb_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_gb_clean[c] = pd.to_numeric(df_fr_clean[c], errors='coerce')

# แปลง datetime
df_gb_clean['publish_time'] = pd.to_datetime(df_gb_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_gb_clean['trending_date'] = pd.to_datetime(df_gb_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_gb_clean['title_length'] = df_gb_clean['title'].astype(str).apply(len)
df_gb_clean['tags_count'] = df_gb_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_gb_clean['days_to_trending'] = (df_gb_clean['trending_date'].dt.normalize() - df_gb_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_gb_clean['weekday'] = df_gb_clean['publish_time'].dt.day_name()
df_gb_clean['hour'] = df_gb_clean['publish_time'].dt.hour

# Save CSV
df_gb_clean.to_csv("df_gb_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_gb_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_gb_clean)}")


ข้อมูล India

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube IN Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/INvideos.csv"


# Read the CSV into df_in

df_in = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_in.printSchema()

# แสดงตัวอย่างข้อมูล
df_in.show(20)


IN Cleaning

In [ ]:
print(f"จำนวนแถว: {df_in.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
# กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_in_clean  = df_in.filter(
    (col("views").cast("string").rlike("^[0-9]+$")) &
    (col("likes").cast("string").rlike("^[0-9]+$")) &
    (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_in_clean =df_in_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_in_clean =df_in_clean.dropDuplicates()

# จัดลำดับตาม views จากน้อยไปมาก
df_in_clean = df_in_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_in_clean.show(20, truncate=False)

# แปลงเป็น Pandas และบันทึกเป็น CSV
df_in_clean = df_in_clean.toPandas()
df_in_clean.to_csv("df_in_clean.csv", index=False)

# จำนวนแถวใน Pandas
print(f"จำนวนแถว: {df_in_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube IN Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/INvideos.csv"

# Read CSV
# --- Clean ด้วย Spark ---
df_in_clean_spark = df_in.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_in_clean = df_in_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_in_clean[c] = pd.to_numeric(df_in_clean[c], errors='coerce')

# แปลง datetime
df_in_clean['publish_time'] = pd.to_datetime(df_in_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_in_clean['trending_date'] = pd.to_datetime(df_in_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_in_clean['title_length'] = df_in_clean['title'].astype(str).apply(len)
df_in_clean['tags_count'] = df_in_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_in_clean['days_to_trending'] = (df_in_clean['trending_date'].dt.normalize() - df_in_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_in_clean['weekday'] = df_in_clean['publish_time'].dt.day_name()
df_in_clean['hour'] = df_in_clean['publish_time'].dt.hour

# Save CSV
df_in_clean.to_csv("df_in_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_in_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_in_clean)}")


ข้อมูล Japan

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube JP Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/JPvideos.csv"


# Read the CSV into df_jp

df_jp = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_jp.printSchema()

# แสดงตัวอย่างข้อมูล
df_jp.show(20)


JP Cleaning


In [ ]:
print(f"จำนวนแถว: {df_jp.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd

# กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_jp_clean = df_jp.filter(
    (col("views").cast("string").rlike("^[0-9]+$")) &
    (col("likes").cast("string").rlike("^[0-9]+$")) &
    (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_jp_clean = df_jp_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_jp_clean = df_jp_clean.dropDuplicates()

#จัดลำดับตาม views จากมากไปน้อย
df_jp_clean = df_jp_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_jp_clean.show(20, truncate=False)

#แปลงเป็น Pandas และบันทึกเป้น CSV
df_jp_clean = df_jp_clean.toPandas()
df_jp_clean.to_csv("df_jp_clean.csv", index=False)

#แสดงจำนวนแถว
print(f"จำนวนแถว: {df_jp_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube JP Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/JPvideos.csv"

# Read CSV
# --- Clean ด้วย Spark ---
df_jp_clean_spark = df_jp.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_jp_clean = df_jp_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_jp_clean[c] = pd.to_numeric(df_jp_clean[c], errors='coerce')

# แปลง datetime
df_jp_clean['publish_time'] = pd.to_datetime(df_jp_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_jp_clean['trending_date'] = pd.to_datetime(df_jp_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_jp_clean['title_length'] = df_jp_clean['title'].astype(str).apply(len)
df_jp_clean['tags_count'] = df_jp_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_jp_clean['days_to_trending'] = (df_jp_clean['trending_date'].dt.normalize() - df_jp_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_jp_clean['weekday'] = df_jp_clean['publish_time'].dt.day_name()
df_jp_clean['hour'] = df_jp_clean['publish_time'].dt.hour

# Save CSV
df_jp_clean.to_csv("df_jp_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_jp_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_jp_clean)}")


In [ ]:
import pandas as pd
from googletrans import Translator

# โหลด CSV
df_jp_clean = pd.read_csv("/content/df_jp_clean.csv")

# เลือก Top trending 100 แถว (ตาม views สูงสุด)
df_jp_clean = df_jp_clean.sort_values(by='views', ascending=False).head(100)

# สร้าง Translator
translator = Translator()

# ฟังก์ชันแปลภาษา พร้อมจัดการ error
def translate_text(text):
    try:
        return translator.translate(str(text), dest='en').text
    except:
        return str(text)

# แปล title
df_jp_clean['title_en'] = df_jp_clean['title'].apply(translate_text)

# บันทึก CSV
df_jp_clean.to_csv("/content/df_jp_clean.csv", index=False)

# ดูตัวอย่างผลลัพธ์
display(df_jp_clean.head(20))

ข้อมูล Korea

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube KR Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/KRvideos.csv"

# Read the CSV into df_kr

df_kr = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_kr.printSchema()

# แสดงตัวอย่างข้อมูล
df_kr.show(20)



KR Cleaning

In [ ]:
print(f"จำนวนแถว: {df_kr.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd

#กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_kr_clean = df_kr.filter(
    (col("views").cast("string").rlike("^[0-9]+$")) &
    (col("likes").cast("string").rlike("^[0-9]+$")) &
    (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_kr_clean = df_kr_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_kr_clean = df_kr_clean.dropDuplicates()

#จัดลำดับตาม views จากมากไปน้อย
df_kr_clean = df_kr_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_kr_clean.show(20, truncate=False)

#แปลงเป็น Pandas และบันทึกเป้น CSV
df_kr_clean = df_kr_clean.toPandas()
df_kr_clean.to_csv("df_kr_clean.csv", index=False)

#แสดงจำนวนแถว
print(f"จำนวนแถว: {df_kr_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube KR Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/KRvideos.csv"

# Read CSV
# --- Clean ด้วย Spark ---
df_kr_clean_spark = df_kr.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_kr_clean = df_kr_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_kr_clean[c] = pd.to_numeric(df_kr_clean[c], errors='coerce')

# แปลง datetime
df_kr_clean['publish_time'] = pd.to_datetime(df_kr_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_kr_clean['trending_date'] = pd.to_datetime(df_kr_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_kr_clean['title_length'] = df_kr_clean['title'].astype(str).apply(len)
df_kr_clean['tags_count'] = df_kr_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_kr_clean['days_to_trending'] = (df_kr_clean['trending_date'].dt.normalize() - df_kr_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_kr_clean['weekday'] = df_kr_clean['publish_time'].dt.day_name()
df_kr_clean['hour'] = df_kr_clean['publish_time'].dt.hour

# Save CSV
df_kr_clean.to_csv("df_kr_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_kr_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_kr_clean)}")


ข้อมูล Mexico


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube MX Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/MXvideos.csv"

# Read the CSV into df_mx

df_mx = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_mx.printSchema()

# แสดงตัวอย่างข้อมูล
df_mx.show(20)



MX Cleaning

In [ ]:
print(f"จำนวนแถว: {df_mx.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd

#กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_mx_clean = df_mx.filter(
    (col("views").cast("string").rlike("^[0-9]+$"))
    & (col("likes").cast("string").rlike("^[0-9]+$"))
    & (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_mx_clean = df_mx.dropna()

# ลบแถวซ้ำทั้งหมด
df_mx_clean = df_mx_clean.dropDuplicates()

#จัดลำดับตาม views จากมากไปน้อย
df_mx_clean = df_mx_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_mx_clean.show(20, truncate=False)

#แปลงเป็น Pandas และบันทึกเป้น CSV
df_mx_clean = df_mx_clean.toPandas()
df_mx_clean.to_csv("df_mx_clean.csv", index=False)

#แสดงจำนวนแถว
print(f"จำนวนแถว: {df_mx_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube MX Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/MXvideos.csv"

# Read CSV
# --- Clean ด้วย Spark ---
df_mx_clean_spark = df_mx.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_mx_clean = df_mx_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_mx_clean[c] = pd.to_numeric(df_mx_clean[c], errors='coerce')

# แปลง datetime
df_mx_clean['publish_time'] = pd.to_datetime(df_mx_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_mx_clean['trending_date'] = pd.to_datetime(df_mx_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_mx_clean['title_length'] = df_mx_clean['title'].astype(str).apply(len)
df_mx_clean['tags_count'] = df_mx_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_mx_clean['days_to_trending'] = (df_mx_clean['trending_date'].dt.normalize() - df_mx_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_mx_clean['weekday'] = df_mx_clean['publish_time'].dt.day_name()
df_mx_clean['hour'] = df_mx_clean['publish_time'].dt.hour

# Save CSV
df_mx_clean.to_csv("df_mx_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_mx_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_mx_clean)}")


ข้อมูล Russian

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YouTube RU Data") \
    .getOrCreate()

# ใส่ path ให้ตรงกับไฟล์ที่แตกไว้
csv_path = "/content/mypath/youtube-new/RUvideos.csv"

# Read the CSV into df_ru

df_ru = spark.read.options(
    header=True,
    inferSchema=True
).csv(csv_path)

# ดู schema
df_ru.printSchema()

# แสดงตัวอย่างข้อมูล
df_ru.show(20)



RU Cleaning

In [ ]:
print(f"จำนวนแถว: {df_ru.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd

#กรองให้ views, likes, dislikes เป็นตัวเลขเท่านั้น
df_ru_clean = df_ru.filter(
    (col("views").cast("string").rlike("^[0-9]+$"))
    & (col("likes").cast("string").rlike("^[0-9]+$"))
    & (col("dislikes").cast("string").rlike("^[0-9]+$"))
)

# ลบแถวที่มีค่าว่างในทุกคอลัมน์
df_ru_clean = df_ru_clean.dropna()

# ลบแถวซ้ำทั้งหมด
df_ru_clean = df_ru_clean.dropDuplicates()

#จัดลำดับตาม views จากมากไปน้อย
df_ru_clean = df_ru_clean.orderBy(col("views").desc())

# ดูตัวอย่างผลลัพธ์
df_ru_clean.show(20, truncate=False)

#แปลงเป็น Pandas และบันทึกเป้น CSV
df_ru_clean = df_ru_clean.toPandas()
df_ru_clean.to_csv("df_ru_clean.csv", index=False)

#แสดงจำนวนแถว
print(f"จำนวนแถว: {df_ru_clean.count()}")

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("YouTube RU Data") \
    .getOrCreate()

# Path CSV
csv_path = "/content/mypath/youtube-new/RUvideos.csv"

# Read CSV
# --- Clean ด้วย Spark ---
df_ru_clean_spark = df_ru.filter(col("title").rlike("[A-Za-z]")) \
    .filter(col("views").rlike("^[0-9]+$") & col("likes").rlike("^[0-9]+$") & col("dislikes").rlike("^[0-9]+$")) \
    .dropna() \
    .dropDuplicates() \
    .orderBy(col("views").desc())

# --- แปลงเป็น Pandas ---
df_ru_clean = df_ru_clean_spark.toPandas()

# แปลงคอลัมน์ตัวเลข
for c in ['views', 'likes', 'dislikes', 'comment_count']:
    df_ru_clean[c] = pd.to_numeric(df_ru_clean[c], errors='coerce')

# แปลง datetime
df_ru_clean['publish_time'] = pd.to_datetime(df_ru_clean['publish_time'], errors='coerce', utc=True).dt.tz_localize(None)
df_ru_clean['trending_date'] = pd.to_datetime(df_ru_clean['trending_date'], errors='coerce', utc=True).dt.tz_localize(None)

# เพิ่มคอลัมน์ใหม่
df_ru_clean['title_length'] = df_ru_clean['title'].astype(str).apply(len)
df_ru_clean['tags_count'] = df_ru_clean['tags'].astype(str).apply(lambda x: 0 if x.strip().lower() == "[none]" else len(x.split("|")))
df_ru_clean['days_to_trending'] = (df_ru_clean['trending_date'].dt.normalize() - df_ru_clean['publish_time'].dt.normalize()).dt.days.astype('Int64')
df_ru_clean['weekday'] = df_ru_clean['publish_time'].dt.day_name()
df_ru_clean['hour'] = df_ru_clean['publish_time'].dt.hour

# Save CSV
df_ru_clean.to_csv("df_ru_ready.csv", index=False)

# ดูตัวอย่างผลลัพธ์
print(df_ru_clean.head(20))

# จำนวนแถว
print(f"จำนวนแถว: {len(df_ru_clean)}")
